In [192]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import librosa
import librosa.display
import IPython.display as ipd

### Classes

In [17]:
df = pd.read_csv('clean_track_info.csv',index_col=0)
df.head()

,track_fma_id,track_spot_id,time_signature,time_sig_confidence,tracks_id,track_name,artist_name,album_name,track_bit,track_top_genre
0,5,66381EvBZ6e3RXzYATpGmN,4,1.000,10,Freeway,Kurt Vile,Constant Hitmaker,192000,Pop
1,18,2ee4Do6eRNXGg7XqKGkvz3,3,1.000,141,Ohio,Alec K. Redfearn & the Eyesores,Every Man For Himself,128000,Folk
2,86,1OrbImTwzIVWtW1msIfa2r,4,0.815,213,Boute,Au,Au,256000,Pop
3,316,4ypiBQX6nsXTwLkaHwnaRI,4,0.349,534,I Can See You,Charles Manson,One Mind,128000,Folk
4,322,4m8jcQJ7wg4RyFNnIvZZgn,3,0.143,540,The Black Pirate,Charles Manson,One Mind,128000,Folk


### Collecting actual audio files for scraped data

In [ ]:
# df['tracks_id'] is the track ID for the audio clips in the fma small dataset

In [99]:
# getting filepaths for the tracks in the df
mp3_file_paths = []
for root, dirs, files in os.walk(r'C:\Users\Mendel Oster\data_science\fma_small\fma_small'):
    for file in files:
        if file.endswith(".mp3"):
            if int(file[:-4]) in list(df['tracks_id'].unique()):
                mp3_file_paths.append(os.path.join(root,file))

In [148]:
# copying the 1222 samples into new directory
for mp3 in mp3_file_paths:
    copy2(mp3, r'C:\Users\Mendel Oster\data_science\fma_small\all_tracks')